In [1]:
import numpy as np 
import pandas as pd 
import statsmodels.api as sm

In [2]:
df=pd.read_csv('/Users/koala/Documents/Lighthub/credit_card_fraud_detection/fraud_reordered.csv')

In [3]:
df.describe()

,Unnamed: 0,is_fraud,cc_num,merch_lat,merch_long,amt,zip,lat,long,city_pop,unix_time
count,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06
mean,9.261965e+05,5.210015e-03,4.173860e+17,3.853898e+01,-9.022794e+01,7.006357e+01,4.881326e+04,3.853931e+01,-9.022783e+01,8.864367e+04,1.358674e+09
std,5.347402e+05,7.199217e-02,1.309115e+18,5.105604e+00,1.375969e+01,1.592540e+02,2.688185e+04,5.071470e+00,1.374789e+01,3.014876e+05,1.819508e+07
min,0.000000e+00,0.000000e+00,6.041621e+10,1.902742e+01,-1.666716e+02,1.000000e+00,1.257000e+03,2.002710e+01,-1.656723e+02,2.300000e+01,1.325376e+09
25%,4.630982e+05,0.000000e+00,1.800429e+14,3.474012e+01,-9.689944e+01,9.640000e+00,2.623700e+04,3.466890e+01,-9.679800e+01,7.410000e+02,1.343017e+09
50%,9.261965e+05,0.000000e+00,3.521417e+15,3.936890e+01,-8.744069e+01,4.745000e+01,4.817400e+04,3.935430e+01,-8.747690e+01,2.443000e+03,1.357089e+09
75%,1.389295e+06,0.000000e+00,4.642255e+15,4.195626e+01,-8.024511e+01,8.310000e+01,7.204200e+04,4.194040e+01,-8.015800e+01,2.032800e+04,1.374581e+09
max,1.852393e+06,1.000000e+00,4.992346e+18,6.751027e+01,-6.695090e+01,2.894890e+04,9.992100e+04,6.669330e+01,-6.795030e+01,2.906700e+06,1.388534e+09


In [4]:
selected_features = ['cc_num', 'amt', 'zip', 'lat', 'long', 'city_pop', 'unix_time', 'merch_lat', 'merch_long']
X = df[selected_features]
y = df['is_fraud']

In [5]:
X=sm.add_constant(X)

model_fit=sm.OLS(y,X).fit()

print(model_fit.summary())

                            OLS Regression Results                            
Dep. Variable:               is_fraud   R-squared:                      -0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    -13.64
Date:                Wed, 24 Jan 2024   Prob (F-statistic):               1.00
Time:                        20:31:22   Log-Likelihood:             2.2456e+06
No. Observations:             1852394   AIC:                        -4.491e+06
Df Residuals:                 1852390   BIC:                        -4.491e+06
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -5.615e-17   2.25e-17     -2.497      0.0

The negative R-squared values and unusual F-statistic might indicate issues with the model, we can tell that it might because of imbalance of dataset.

The steps we took next is to resample with sample size 10000 with 50% fraudulent activities and 50% non-fraud actvities.

In [6]:
from sklearn.utils import resample

In [10]:
# Assuming 'df' is your DataFrame with relevant columns including 'is_fraud'
selected_features = ['cc_num', 'amt', 'zip', 'lat', 'long', 'city_pop', 'unix_time', 'merch_lat', 'merch_long']
X = df[selected_features]
y = df['is_fraud']

# Separate the data into non-fraud and fraud instances
non_fraud = df[df['is_fraud'] == 0]
fraud = df[df['is_fraud'] == 1]

# Sample 5000 instances for each class
non_fraud_sampled = resample(non_fraud, replace=False, n_samples=5000, random_state=42)
fraud_sampled = resample(fraud, replace=False, n_samples=5000, random_state=42)

# Combine the sampled data
df_balanced = pd.concat([non_fraud_sampled, fraud_sampled],ignore_index=True)


In [11]:
# redo the regression model
selected_features = ['cc_num', 'amt', 'zip', 'lat', 'long', 'city_pop', 'unix_time', 'merch_lat', 'merch_long']
X = df_balanced[selected_features]
y = df_balanced['is_fraud']

In [12]:
X=sm.add_constant(X)

model_fit=sm.OLS(y,X).fit()

print(model_fit.summary())

                            OLS Regression Results                            
Dep. Variable:               is_fraud   R-squared:                      -0.003
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                    -9.144
Date:                Wed, 24 Jan 2024   Prob (F-statistic):               1.00
Time:                        20:37:26   Log-Likelihood:                -7271.7
No. Observations:               10000   AIC:                         1.455e+04
Df Residuals:                    9996   BIC:                         1.458e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.734e-15   1.39e-15     -1.247      0.2

The negative R-squared values, unusual F-statistic, and the fact that many coefficients are very close to zero with high p-values suggest that the model and the individual coefficients are not statistically significant.